# RSA 🕵️️

En este Notebook implementaremos el sistema de criptografía asimétrico RSA descrito en la [Pregunta 2](https://github.com/UC-IIC3253/2021/blob/main/tareas/tarea2/enunciado.pdf) de la Tarea 2 del curso IIC3253 Criptografía y Seguridad Computacional (v.2021-1).

Elaborado por: Vicente Merino

Github: [VicenteMerino](https://github.com/VicenteMerino)

## 1. Importación de librerías 📚

Primero importaremos la función `randint` de la librería `random` que utilizaremos en algunas partes del algoritmo para obtener números aleatorios en un rango dado.

In [1]:
from random import randint

## 2. Exponenciación rápida 🚀

Implementamos dos algoritmos, que nos permitirán calcular de forma mucho más eficiente la exponenciación $(a^b)$ y la exponenciación con módulo $(a^b\;\text{mod}\;n)$

In [2]:
def fast_exp(a: int, b: int) -> int:
  if a < 0 or b < 0:
    raise Exception('Invalid parameters values')
  if a == 0 and b == 0:
    raise Exception('Invalid parameters value (0 to 0 exponentiation)')
  
  if b == 0:
    return 1
  elif b % 2 == 0:
    t = fast_exp(a, b // 2)
    return t*t
  else:
    t = fast_exp(a, (b-1) // 2)
    return t*t*a

def exp_mod(a: int, b: int, n: int) -> int:
  if a < 0 or b < 0 or n <= 0:
    raise Exception('Invalid parameters values')
  if a == 0 and b == 0:
    raise Exception('Invalid parameters value (0 to 0 exponentiation)')

  if b == 0:
    return 1
  elif b % 2 == 0:
    t = exp_mod(a, b // 2, n)
    return (t*t) % n
  else:
    t = exp_mod(a, (b-1) // 2, n)
    return (t*t*a) % n


## 3. Test de primalidad y generar primo 🙋

Implementamos el test de primalidad de Miller-Rabin, que nos permite determinar si un número $n$ es primo o no con probabilidad $2^{-k}$. A partir de esto, implementamos una función que nos permite encontrar un número primo 

In [3]:
def miller_rabin(n: int, k: int) -> bool:
  if n < 1 or k < 1:
    raise Exception('Invalid parameters values')
  if n == 2 or n == 3:
    return True
  m = n - 1
  l = 0
  while m % 2 == 0:
    l += 1
    m = m//2

  for i in range(k):
    b = randint(2, n - 2)
    x = exp_mod(b, m, n)
    if x == 1 or x == n - 1:
      continue
    continue_bool = False
    for j in range(l - 1):
      x = exp_mod(x, 2, n)
      if x == n - 1:
        continue_bool = True
        break
    if continue_bool:
      continue
    return False
  return True

def generar_primo(l: int) -> int:
  if l < 1:
    raise Exception('Invalid parameter value')
  min_value = fast_exp(10, l-1)
  max_value = fast_exp(10, l)-1

  random_number = randint(min_value, max_value)

  while not miller_rabin(random_number, 100):
    random_number = randint(min_value, max_value)
  return random_number


## 4. Algoritmo extendido de euclides ➗

Implementamos el algoritmo extendido de euclides, que nos permite encontrar el máximo común divisor entre dos números y los multiplicadores de cada uno.

In [4]:
def alg_ext_euclides(a: int, b: int) -> (int, int, int):
  if a < b or b < 0 or a <= 0:
    raise Exception('Invalid parameters values')
  def recursion(n1: int, n2: int) -> (int, int, int):
    if n1 == 0:
      return (n2, 0, 1)

    mcd = recursion(n2 % n1, n1)

    s0 = mcd[1]
    t0 = mcd[2]

    s = t0 - n2//n1 * s0
    t = s0

    return (mcd[0], s, t)
  return recursion(a, b)

## 5. Función inverso modular ↕️

Implementamos una función que nos permite encontrar el inverso en módulo $n$ de un número $a$.

In [5]:
def inverso(a: int, n: int) -> int:
  if a < 1 or n < 2:
    raise Exception('Invalid parameters values')
  
  if a < n:
    mcd = alg_ext_euclides(n, a)
    if mcd[0] != 1:
      raise Exception('The numbers are not relative primes')
    if mcd[2] >= 0:
      return mcd[2]
    else:
      return mcd[2] + n
  else:
    mcd = alg_ext_euclides(a, n)
    if mcd[0] != 1:
      raise Exception('The numbers are not relative primes')
    if mcd[1] >= 0:
      return mcd[1]
    else:
      return mcd[1] + n

## 6. Generación de claves 🗝️️

Implementamos una función para generar las claves públicas y privadas del sistema RSA y las guardamos en los archivos `public_key.txt` y `private_key.txt`.

In [6]:
def generar_clave(l: int):
  if l < 0:
    raise Exception("Length of the keys must be positive")
  P = generar_primo(l)
  Q = generar_primo(l)
  while P == Q:
    Q = generar_primo(l)
  N = P*Q
  Phi_N = (P - 1)*(Q - 1)

  d = randint(0, Phi_N - 1)

  mcd = alg_ext_euclides(Phi_N, d)
  while mcd[0] != 1:
    d = randint(0, Phi_N - 1)
    mcd = alg_ext_euclides(Phi_N, d)

  e = inverso(d, Phi_N)

  with open("private_key.txt", "w") as private_file:
    private_file.write(str(d))
    private_file.write("\n")
    private_file.write(str(N))

  with open("public_key.txt", "w") as public_file:
    public_file.write(str(e))
    public_file.write("\n")
    public_file.write(str(N))

## 7. Encriptación y decriptación 🐱‍💻

Funciones que leen las llaves públicas y privadas en `public_key.txt` y `private_key.txt` y con ellas encriptan/decriptan un mensaje. 

In [7]:
def enc(m: int) -> int:
  with open("public_key.txt", "r") as public_file:
    lines = public_file.readlines()
    e = int(lines[0])
    N = int(lines[1])
  return exp_mod(m, e, N)

def dec(m: int) -> int:
  with open("private_key.txt", "r") as private_file:
    lines = private_file.readlines()
    d = int(lines[0])
    N = int(lines[1])
  return exp_mod(m, d, N)

## 8. Testing 🧪

Probamos encriptar y desencriptar distintos mensajes aleatorios. Para ello creamos dos _test cases_, uno con mensajes con la misma llave, y otro con llaves distintas para cada mensaje.

In [8]:
def same_key_test_case(l):
  generar_clave(l)
  for _ in range(1000):
    m = randint(0, 2**100)
    if m != dec(enc(m)):
      return False
  return True


def different_key_test_case(l):
  for _ in range(100):
    generar_clave(l)
    m = randint(0,2**100)
    if m != dec(enc(m)):
      return False
  return True


print(f"Same key test case: {same_key_test_case(100)}")
print(f"Different key test case: {different_key_test_case(100)}")

Same key test case: True
Different key test case: True
